In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import time


In [13]:
# Define the years we plan to download from CalGEM
years = ['2020','2021']

# Set pandas to display all columns
pd.set_option('display.max_columns', None)

# Load the CSV file into a pandas dataframe
df_fieldcode = pd.read_csv('FieldCode.csv')

# Iterate over each year we plan to download.
for year in years:
    print(f'Currently processing year:({year})')

    # Create a dictionary of filenames for each dataset we plan to work with.
    calgem_filenames = {
        "wells_file": f"{year}CaliforniaOilAndGasWells.csv",
        "production_file": f"{year}CaliforniaOilAndGasWellMonthlyProduction.csv"
    }

    # Construct the CalGEM urls needed to download the datasets.
    wells_url = f"https://calgem-pid.conservation.ca.gov/pid/{calgem_filenames['wells_file']}"
    production_url = f"https://calgem-pid.conservation.ca.gov/pid/{calgem_filenames['production_file']}"
  
    # step to be able to webscrape the data: https://stackoverflow.com/questions/55711159/pandas-read-csv-from-url-and-include-request-header
    # Custom way to tell CalGEM to accept our request for the data.
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'
    }

    # Create a dataframe for each dataset from CalGEM, and formats the data as CSV.
    df_wells_data = pd.read_csv(wells_url, storage_options=headers)
    df_production_data = pd.read_csv(production_url, storage_options=headers)

    # add column Year to df_wells_data in each year downloaded.
    df_wells_data['ReportYear'] = year
    df_production_data['ReportYear'] = year    
 
    # First join: Merge df_fieldcode with df_production_data on 'FIELD_CODE' and 'FieldCode'
    df_merged_1 = df_production_data.merge(df_fieldcode, left_on='FieldCode', right_on='FIELD_CODE', how='left')

    # Second join: Merge the result with df_wells_data on 'APINumber' and 'API'
    df_final_merged = df_merged_1.merge(df_wells_data, left_on='APINumber', right_on='API', how='left', suffixes=('_production', '_wells'))

    # Filter rows where 'ReportedOrEstimated' equals a specific value, e.g., 'Reported'
    df_filtered = df_final_merged[df_final_merged['ReportedOrEstimated'] == 'Reported']

    # Or filter rows where 'ReportedOrEstimated' is not NaN (if you want to exclude missing values)
    # df_filtered = df_final_merged[df_final_merged['ReportedOrEstimated'].notna()]

    # Write the filtered dataframe to a CSV file
    df_filtered.to_csv(f'{year}_filtered_final_output.csv', index=False)

    # Preview the first 5 rows of the final merged dataframe
    print(f"Preview of fully merged data for {year}:")
    print(df_final_merged.head())

    # Completed this year. Waiting 5 seconds before we process the next years worth of data.
    # print(f'Processing year complete:({year}), waiting 5 sec...')
    # time.sleep(5)

# We are done.
print("Download Complete.")

Currently processing year:(2020)
Preview of fully merged data for 2020:
  ReportType    APINumber  FieldCode_production  AreaCode_production  \
0      OG110  40293867900                   464                    0   
1      OG110  42372032701                   849                   45   
2      OG110  42372032701                   849                   45   
3      OG110  40305619700                    52                    0   
4      OG110  40293525700                   464                    0   

   PoolCode_production WellTypeCode_production ProductionReportDate  \
0                    0                      OG  2020-05-31 00:00:00   
1                   35                     NaN  2020-12-31 00:00:00   
2                   35                     NaN  2020-12-31 00:00:00   
3                   20                     NaN  2020-02-29 00:00:00   
4                    0                      SC  2020-12-31 00:00:00   

   ProductionStatus  CasingPressure  TubingPressure  BTUofGasProduce